## Moat Brand Reporting Refresh

In [1]:
from rtf_utils.moat_utils import MoatTile
from essence.analytics.platform import securedcredentials as secure_creds
import warnings

warnings.filterwarnings("ignore", "Your application has authenticated using end user credentials")
import json

In [12]:
service_account_email='131786951123-compute@developer.gserviceaccount.com' ## gcp analytics service account
PROJECT_ID='essence-analytics-dwh'

credentialsFromVault=secure_creds.getCredentialsFromEssenceVault(service_account_email)

moat_token = secure_creds.getDataFromEssenceVault('Moat_Token_Google')

In [13]:
start_date = '20190701'
end_date = '201909012'

In [4]:
with open('dags/brand_report_task_queue.json') as json_file:
    config_json = json.load(json_file)
    campaigns = config_json['campaigns']

In [14]:
ig_vid = MoatTile(tile_id=13120,
         tile_type="vid",name = "ig_vid",
         level_filters={'level1':10154328017481183},
         dimensions=['date','level4'])

yt_trv = MoatTile(tile_id=13386,
         tile_type="vid",name = "YT_TRV",
         level_filters={'level1':7020493427},
         dimensions=['date','level4'])

In [2]:
ig_vid = MoatTile(tile_id=13120)

In [ ]:
ig_vid.get_data()